In [37]:
from scipy import stats
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal as Normal

In [2]:
fig, ax = plt.subplots(1, 1)
plt.title("Probability Densitiy Function")
plt.xlabel("Distance from Origin")
plt.ylabel("Probability Density.")

In [3]:
dfs = range(1,5) + range(10,70,20) + [100]
for df in dfs:
    x = np.linspace(stats.chi.ppf(0.01, df), stats.chi.ppf(0.99, df), 100)
    ax.plot(x, stats.chi.pdf(x, df), alpha=0.6, label='D = {}'.format(df))
    ax.legend(loc='best', frameon=True)

In [4]:
fig.savefig("hw2_chi_pdf")

In [5]:
fig, ax = plt.subplots(1, 1)
plt.title("Cumulative Distribution Function")
plt.xlabel("Upper Bound on Distance From Origin")
plt.ylabel("Probability.")
for df in [1,5,100]:
    x = np.linspace(0, 12, 100)
    ax.plot(x, stats.chi.cdf(x, df), alpha=0.6, label='D = {}'.format(df))
    ax.legend(loc='best', frameon=True)

In [6]:
fig.savefig("hw2_chi_cdf")

In [7]:
# Problem 2
fig, ax = plt.subplots(1, 1)
plt.title("Data Points")
plt.xlabel("X values")
plt.ylabel("Y values")
x = [-1.87, -1.76, -1.67, -1.22, -0.07, 0.11, 0.67, 1.60, 2.22, 2.51]
y = [0.06, 1.67, 0.54, -1.45, -0.18, -0.67, 0.92, 2.95, 5.13, 5.18]
plt.scatter(x,y)

In [8]:
# We center the data and plot the points again.
fig, ax = plt.subplots(1, 1)
plt.title("Centered Data Points")
plt.xlabel("X values")
plt.ylabel("Y values")
xbar = x - np.mean(x)
plt.scatter(xbar,y)

In [9]:
# Now we'll be plotting several random functions drawn from my prior.
lb, ub = min(x), max(x)
xaxis = np.linspace(lb-0.1, ub+0.1, 100)

In [10]:
def samplePriorWeights(w0, V0):
    '''
    w0 is the mean and V0 the covariance.
    '''
    return Normal(w0, V0)

In [11]:
plt.show()

In [12]:
def designMatrix(points):
    return np.array([ [1, point, point**2, point**3, point**4] for point in points])


In [13]:
# Function to plot prior
w0 = [-0.5, 0.1, 0.1, 0, 0 ]
V0 = 0.1 * np.identity(5)
sigma2 = 0.5
def plotPrior(xs):
    mat = designMatrix(xs)
    w = samplePriorWeights(w0, V0)
    #print w
    epsilon = Normal(np.zeros(len(xs)), sigma2 * np.identity(len(xs)))
    #print epsilon
    return mat.dot(w) + epsilon

In [14]:
fig, ax = plt.subplots(1, 1)
plt.title("Random Prior Samples")
plt.xlabel("X values")
plt.ylabel("Y values")
colors = plt.cm.rainbow(range(10))
for color in colors:
    plt.scatter(xaxis,plotPrior(xaxis), color=color)

In [15]:
xbar = x - np.mean(x)
plt.scatter(xbar,y)
plt.show()

In [16]:
def plotPosterior(train, ytrain, xs, s2, V0, w0):
    X = designMatrix(train)
    Xplot = designMatrix(xs)
    Vnew = s2* np.linalg.inv(s2 * np.linalg.inv(V0) + X.T.dot(X))
    # print Vnew.shape
    # print X.shape
    wnew = Vnew.dot(V0).dot(w0) + (1.0 / s2) * Vnew.dot(X.T).dot(ytrain)
    w = Normal(wnew, Vnew)
    return Xplot.dot(w)

In [17]:
fig, ax = plt.subplots(1, 1)
plt.title("Posterior Samples")
plt.xlabel("X values")
plt.ylabel("Y values")
colors = plt.cm.rainbow(range(5))
for color in colors:
    plt.scatter(xaxis, plotPosterior(xbar, y, xaxis, sigma2, V0, w0), color=color)

In [18]:
xbar = x - np.mean(x)
plt.scatter(xbar,y)
plt.show()

In [19]:
def plotPredictive(train, ytrain, xstar, s2, V0, w0):
    X = designMatrix(train)
    xstarphi = designMatrix([xstar])[0]
    Vnew = s2* np.linalg.inv(s2 * np.linalg.inv(V0) + X.T.dot(X))
    wnew = Vnew.dot(V0).dot(w0) + (1.0 / s2) * Vnew.dot(X.T).dot(ytrain)
    mean = xstarphi.dot(wnew)
    cov = s2 + xstarphi.dot(Vnew).dot(xstarphi.T)
    
    # Calculate PDF.
    low = stats.norm.ppf(0.025, loc = mean, scale=cov)
    high = stats.norm.ppf(0.975, loc= mean, scale=cov)
    return (low, mean, high)

In [20]:
fig, ax = plt.subplots(1, 1)
plt.title("Predictive Samples and 95% Confidence Interval")
plt.xlabel("X values")
plt.ylabel("Y values")
alls = [plotPredictive(xbar, y, xstar, sigma2, V0, w0) for xstar in xaxis]
low, middle, high = zip(*alls)
plt.scatter(xaxis, low, color='red')
plt.scatter(xaxis, middle)
plt.scatter(xaxis, high, color='red')

In [21]:
xbar = x - np.mean(x)
plt.scatter(xbar,y, color='green')
plt.show()

In [319]:
def marginalLikeliHood(ytest, X, w0, V0, s2):
    mean = X.dot(w0)
    cov = s2 * np.identity(len(X))  + X.dot(V0).dot(X.T)
    return stats.multivariate_normal.pdf(ytest, mean=mean, cov=cov)

In [341]:
designMatrices = []
designMatrices.append(np.array([ [1, np.sin(p), np.sin(2*p), np.sin(3*p), np.sin(4*p)] for p in xbar]))
designMatrices.append(np.array([ [1, np.cos(p), np.cos(2*p), np.cos(3*p), np.cos(4*p)] for p in xbar]))
designMatrices.append(np.array([ [1.0 / (p**2), 1.0/p, 1, p, p**2] for p in xbar]))
designMatrices.append(np.array([ [1, p**2, p**4, p**6, p**8] for p in xbar]))
designMatrices.append(np.array([ [1, p, p**2, p**3, p**4] for p in xbar]))
designMatrices.append(np.array([ [0, 0, 0, 1, p**4] for p in xbar]))
designMatrices.append(np.array([ [0, 0, p, p**2, p**4] for p in xbar]))
designMatrices.append(np.array([ [0, 0, 0, p**2, p**4] for p in xbar]))

In [344]:
# Plot the above
fig, ax = plt.subplots()
ps = np.array([marginalLikeliHood(y, m, w0, V0, sigma2) for m in designMatrices])
ax.bar(np.arange(8), ps / sum(ps) , 1)

<Container object of 8 artists>

In [345]:
plt.title("Model Comparison")
plt.ylabel("Normalized Probability")
plt.xlabel("Model")
ax.set_xticks(np.arange(8)+1)
ax.set_xticklabels( ('Sine', 'Cosine', 'Inverse', 'Poly 1', 'Poly 2', 'Poly 3', 'Poly 4','Poly 5') )
plt.show()

In [371]:
# Read in the data
train = np.genfromtxt('spam.train.dat', delimiter=" ")
test = np.genfromtxt('spam.test.dat', delimiter=" ")
Xtest = test[:, :-1]
Ytest = test[:, -1]
Xtrain = train[:, :-1]
Ytrain = train[:, -1]

In [280]:
# Problem 3
def negativeLogMap(X,y,w,s2):
    '''
    Using Bayes Rule, calculate a MAP estimate.
    '''
    denom = -np.logaddexp(0, -X.dot(w))
    return -np.sum(-y*denom + (1-y) * (X.dot(w) + denom)) + w.dot(w) / (2 * s2)

In [281]:
def logMAPGradient(X,y,w,s2):
    '''
    Gradient of the NLL.
    '''
    return X.T.dot(1.0 / (1.0 + np.exp(-X.dot(w))) - y) +  w / s2

In [282]:
def minNegLogMAP(x):
    return negativeLogMap(Xtrain, Ytrain, x, 1)
res = optimize.minimize(minNegLogMAP, np.zeros(len(Xtrain[0])), method='CG')
wPred = res.x

In [283]:
def negLogMAPGrad(w, sigma2):
    return logMAPGradient(Xtrain, Ytrain, w, sigma2)

In [269]:
res2 = optimize.root(lambda x: negLogMAPGrad(x, 1), np.zeros(57))
wGradPred = res2.x

In [238]:
wGradPred

array([  1.84208015e-07,  -3.05912545e-07,   4.17827134e-07,
         4.04982554e-07,   6.50426977e-07,   2.97057812e-07,
         5.65705267e-07,   3.57786901e-07,   3.72966145e-07,
         3.92136580e-07,   2.22415084e-07,  -1.13013472e-07,
         1.85309744e-07,   1.42145275e-07,   2.66990381e-07,
         9.95597124e-07,   5.24629346e-07,   4.40472991e-07,
         1.60777366e-06,   4.28388021e-07,   1.87427724e-06,
         4.65372868e-07,   5.99015172e-07,   4.52477723e-07,
        -2.41873822e-06,  -1.26165129e-06,  -3.81241829e-06,
        -5.13742422e-07,  -4.87328085e-07,  -4.66524227e-07,
        -2.99094517e-07,  -2.12830769e-07,  -3.85757405e-07,
        -2.11866738e-07,  -4.59441127e-07,  -3.20890963e-07,
        -4.49439620e-07,  -5.10721426e-08,  -2.88639125e-07,
        -1.54803028e-07,  -2.12594415e-07,  -6.76247808e-07,
        -1.83258491e-07,  -4.04808828e-07,  -9.58185197e-07,
        -7.43167965e-07,  -2.34532643e-08,  -1.33182350e-07,
        -9.40865917e-08,

In [291]:
def verifyGradiant(x, epsilon = 0.0001):
    """
    Uses finite differences to verify the gradiant of our log posterior.
    """
    units = np.diag([epsilon for _ in x])
    grad = negLogMAPGrad(x,1)
    finiteDiff = [(minNegLogMAP(x + units[i]) - minNegLogMAP(x - units[i])) / (2*epsilon) for i in xrange(len(x))]
    return all([abs(g - f) < 0.01 for g,f in zip(grad, finiteDiff)])

In [292]:
# Verify the result for ntrials
def verifyGradiantM(ntrials):
    xs = [np.random.rand(57) for _ in xrange(ntrials)]
    return all(map(verifyGradiant, xs))

In [293]:
print verifyGradiantM(1)

False


In [294]:
def RMSE(X,Y, w):
    import math
    return math.sqrt(np.sum((Y - (1.0 / (1 + np.exp(-X.dot(w)))))**2) / float(len(X)))

def kCrossValidation(k, sigma2, data=Xtrain, ydata=Ytrain, testdata=Xtest, ytestdata=Ytest):
    # Split original data into k section
    splits = [len(Xtrain) *i / k for i in range(k)] + [len(Xtrain)]
    print splits
    
    results = []
    testResults = []
    for i in range(k):
        test = data[splits[i]:splits[i+1], :]
        ytest = ydata[splits[i]:splits[i+1]]
        train = data[range(splits[i]) + range(splits[i+1],len(data)), :]
        ytrain = ydata[range(splits[i]) + range(splits[i+1], len(data))]
        opt = optimize.root(lambda x : logMAPGradient(train,ytrain,x,sigma2), np.zeros(57))
        wPredicted = opt.x
        results.append(RMSE(test, ytest, wPredicted))
        testResults.append(RMSE(testdata, ytestdata, wPredicted))
    return np.mean(results), np.mean(testResults)

In [295]:
logsigmas = range(-8,9)
results = [kCrossValidation(10, np.exp(s)) for s in logsigmas]

[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200,

In [303]:
print results
resultsCross, resultsTest = zip(*results)
tplot = plt.scatter(logsigmas, resultsTest, color="red")
cplot = plt.scatter(logsigmas, resultsCross, color="green")
plt.legend((tplot, cplot),
           ('Test RMSE', '10-fold Cross Validation RMSE'),
           scatterpoints=1,
           loc='lower left',
           fontsize=8)
plt.xlabel("log(sigma^2)")
plt.ylabel("Root Mean Square Error")

[(0.42988064794329095, 0.42702964826920242), (0.39903489018491339, 0.39485359170268886), (0.36476697494759003, 0.3590933056575446), (0.33195738145393172, 0.32559816393190244), (0.3040903350606789, 0.29889028753046698), (0.28284370650712004, 0.28017394146316593), (0.26874064259272712, 0.26826900132836218), (0.26082770904443203, 0.2614316279525129), (0.25712354566033213, 0.25787185204932445), (0.25587891836652793, 0.25634039361499406), (0.25596794326005851, 0.2560312818878886), (0.25651922726898491, 0.25597677038841055), (0.25697832582888391, 0.25588688246819569), (0.25744531020011502, 0.25597611355393302), (0.25773243960063463, 0.25616613387000886), (0.25793847610773424, 0.2563603762371981), (0.25813492153915324, 0.25651051606402137)]


In [304]:
plt.show()

In [298]:
# We now perform some standardization on the data.
def normalize_features(X_train, X_test):
    mean_X_train = np.mean(X_train, 0)
    std_X_train = np.std(X_train, 0)
    std_X_train[ std_X_train == 0 ] = 1
    X_train_normalized = (X_train - mean_X_train) / std_X_train
    X_test_normalized = (X_test - mean_X_train) / std_X_train
    
    return X_train_normalized, X_test_normalized

In [305]:
normXtrain, normYtrain = normalize_features(Xtrain.T, Ytrain)
normXtest, normYtest = normalize_features(Xtest.T, Ytest)

In [312]:
logsigmas = range(-8,9)
results2 = [kCrossValidation(10, np.exp(s), normXtrain.T, normYtrain, normXtest.T, normYtest) for s in logsigmas]

[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200,

In [313]:
print results2
resultsCross, resultsTest = zip(*results2)
tplot = plt.scatter(logsigmas, resultsTest, color="red")
cplot = plt.scatter(logsigmas, resultsCross, color="green")
plt.legend((tplot, cplot),
           ('Test RMSE for Normalized Data', '10-fold Cross Validation RMSE for Normalized Data'),
           scatterpoints=1,
           loc='lower left',
           fontsize=8)
plt.xlabel("log(sigma^2)")
plt.ylabel("Root Mean Square Error")

[(0.18157300962484479, 0.182819300899488), (0.17592907408096675, 0.17773147536257672), (0.17561907741104593, 0.17736306672233398), (0.17561675802808993, 0.17735942576862632), (0.17561675799042892, 0.17735942569202703), (0.17561675799042892, 0.17735942569202703), (0.17561675799042892, 0.17735942569202703), (0.17561675799042892, 0.17735942569202703), (0.17589285423105258, 0.17752437166209128), (0.17840243915907006, 0.17846008058962637), (0.17561499739147518, 0.17736320554130716), (0.17563383360432921, 0.17737256909649007), (0.1756334970870323, 0.17737843773116532), (0.17563529363966413, 0.17739160202926002), (0.17561459748521646, 0.17737586048520049), (0.17566815767993754, 0.17742838467015348), (0.17568142963501993, 0.17744409985126636)]


In [314]:
plt.show()

In [317]:
def negLogMAPGrad(w, sigma2):
    return logMAPGradient(normXtrain.T, normYtrain.T, w, sigma2)

res3 = optimize.root(lambda x: negLogMAPGrad(x, np.exp(1)), np.zeros(57))
wGradPred = res3.x

In [318]:
wGradPred

array([  3.3239818 ,   3.7805773 ,   3.96723509,   3.24682572,
         1.812309  ,   3.37176343,   2.69858099,   3.26723951,
         3.21053188,   3.57863507,   2.93518846,   5.80321697,
         3.20148729,   3.3682881 ,   3.0651955 ,   2.26639966,
         3.18783952,   3.53772384,   0.33443072,   3.02108896,
         0.12313008,   3.25544495,   2.88668557,   3.21139298,
        -5.3504362 ,  -2.05567011,   0.32648644,   3.41915223,
         3.73286432,   3.4745214 ,   3.45743104,   3.119373  ,
         3.55698906,   3.12266529,   3.56520895,   3.4818673 ,
         3.41311887,   2.34897263,   3.38181961,   3.17374505,
         3.33515686,  -1.45265659,   3.2920016 ,   1.80916005,
       -13.28127109,  -1.66904174,   3.29165059,   3.27698537,
         3.15808291,   3.72040009,   3.29212317,   0.5787002 ,
         2.74064018,   3.1665611 ,  -5.66067519,  -0.61890323,  -0.29541735])

In [357]:
for i in range(len(Xtrain[0])):
    print Xtrain[:, i].max(), Xtrain[:, i].var()

4.34 0.0834651727889
14.28 1.5602771744
4.54 0.240965132622
42.73 1.80279276227
9.09 0.451633559722
3.44 0.0618669617667
7.27 0.157046604167
11.11 0.166051257233
5.26 0.0783172257333
18.18 0.489108013156
2.06 0.0370296441667
9.67 0.752641730833
2.94 0.0705337975
10.0 0.111589678889
4.41 0.0695421006222
20.0 0.712749705556
7.14 0.188993064167
7.69 0.281013398933
18.75 3.13527580582
6.32 0.169593815289
11.11 1.40176137146
17.1 0.940210426667
4.76 0.124112466656
12.5 0.181356745556
20.83 2.78818950116
16.66 0.862432956933
33.33 11.5294699747
9.09 0.3067355431
14.28 0.390630881956
5.88 0.207423745556
12.5 0.1728945879
4.76 0.0987427327889
18.18 0.348104303456
4.76 0.0990575386222
20.0 0.319134094656
7.69 0.1580419216
6.89 0.173081322656
8.33 0.0596935997222
9.75 0.168894287289
4.76 0.1088402599
7.14 0.145509982222
14.28 0.594633412933
3.57 0.0529594244333
20.0 0.505215981289
21.42 1.09504480746
16.7 0.718548041233
2.17 0.00758992333333
5.0 0.0518996837333
4.385 0.0652787094093
4.271 0.0505

In [372]:
def transform(X):
    def binary(x):
        return 0 if x == 0 else 1
    newX = X
    vecbinary = np.vectorize(binary)
    newX[:, 48:52] = vecbinary(X[:, 48:52])
    return newX

In [373]:
newnormXtrain, newnormYtrain = normalize_features(transform(Xtrain.T), Ytrain)
newnormXtest, newnormYtest = normalize_features(transform(Xtest.T), Ytest)

In [375]:
logsigmas = range(-8,9)
results3 = [kCrossValidation(10, np.exp(s), newnormXtrain.T, newnormYtrain, newnormXtest.T, newnormYtest) for s in logsigmas]

[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
[0, 300, 600, 900, 1200,

In [376]:
print results3
resultsCross, resultsTest = zip(*results3)
tplot = plt.scatter(logsigmas, resultsTest, color="red")
cplot = plt.scatter(logsigmas, resultsCross, color="green")
plt.legend((tplot, cplot),
           ('Test RMSE for Normalized Data', '10-fold Cross Validation RMSE for Normalized Data'),
           scatterpoints=1,
           loc='lower left',
           fontsize=8)
plt.xlabel("log(sigma^2)")
plt.ylabel("Root Mean Square Error")

[(0.18651052232593085, 0.18483014094664468), (0.18098988295441615, 0.17930605224146728), (0.18067603673932392, 0.17893114394623441), (0.18067353693764504, 0.17892752415429475), (0.18067353689641594, 0.17892752407853307), (0.18067353689641594, 0.17892752407853307), (0.18067353689641594, 0.17892752407853307), (0.18067353689641594, 0.17892752407853307), (0.1809251404017409, 0.17892888504753701), (0.18400286286866507, 0.18015614165276689), (0.18067208981993044, 0.17893148181124116), (0.18069460121907113, 0.17895124298453952), (0.18069133989996297, 0.17895241355016561), (0.18069115874743996, 0.1789511443015126), (0.18070665923902843, 0.17895189494095046), (0.18076397066463695, 0.17896000358540104), (0.18092194403195205, 0.17926259089030944)]


In [377]:
plt.show()